# Multi-Agent Workflow with Brave Search API for Grounding

This notebook demonstrates a production-ready multi-agent system that uses Brave Search API for grounding. The system includes:

- **Research Agent**: Gathers information from multiple searches
- **Fact-Checking Agent**: Verifies claims using search results
- **Analysis Agent**: Analyzes and synthesizes information
- **Quality Assurance Agent**: Ensures response accuracy and completeness
- **Orchestrator**: Coordinates agent interactions

## Key Features
- Real-time web search grounding
- Multi-agent collaboration with LangGraph
- Fact verification and citation tracking
- Iterative refinement based on search results
- Comprehensive error handling and retry logic

## 1. Installation and Setup

In [ ]:
# Install required packages
!pip install -q langgraph langchain langchain-openai langchain-anthropic brave-search python-dotenv pydantic typing-extensions rich pandas numpy

In [ ]:
import os
import json
import asyncio
from typing import Dict, List, Any, Optional, TypedDict, Annotated, Sequence, Literal
from datetime import datetime, timezone
from dataclasses import dataclass, field
from enum import Enum
import logging
from functools import lru_cache

# LangChain and LangGraph imports
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages

# Brave Search
from brave_search import BraveSearch

# Pydantic for data validation
from pydantic import BaseModel, Field, validator

# Rich for beautiful output
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.syntax import Syntax
from rich.progress import track

console = Console()

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## 2. Environment Configuration

In [ ]:
# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Configuration
BRAVE_API_KEY = os.getenv("BRAVE_API_KEY", "YOUR_BRAVE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "YOUR_OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY", "YOUR_ANTHROPIC_API_KEY")

# Set API keys
os.environ["BRAVE_API_KEY"] = BRAVE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

console.print(Panel("[green]✓ Environment configured[/green]", title="Setup Status"))

## 3. Brave Search API Client

In [ ]:
import requests
from typing import Optional, Dict, List, Any
import time

class BraveSearchClient:
    """Enhanced Brave Search API client with retry logic and caching"""
    
    def __init__(self, api_key: str, max_retries: int = 3, cache_ttl: int = 3600):
        self.api_key = api_key
        self.base_url = "https://api.search.brave.com/res/v1"
        self.max_retries = max_retries
        self.cache_ttl = cache_ttl
        self.cache = {}
        self.headers = {
            "Accept": "application/json",
            "X-Subscription-Token": self.api_key
        }
    
    def web_search(self, 
                   query: str, 
                   count: int = 10,
                   freshness: Optional[str] = None,
                   country: str = "us",
                   search_lang: str = "en",
                   ui_lang: str = "en-US",
                   text_decorations: bool = False,
                   spellcheck: bool = True) -> Dict[str, Any]:
        """Execute web search with Brave Search API"""
        
        # Check cache
        cache_key = f"{query}_{count}_{freshness}_{country}"
        if cache_key in self.cache:
            cached_result, timestamp = self.cache[cache_key]
            if time.time() - timestamp < self.cache_ttl:
                logger.info(f"Cache hit for query: {query}")
                return cached_result
        
        params = {
            "q": query,
            "count": count,
            "country": country,
            "search_lang": search_lang,
            "ui_lang": ui_lang,
            "text_decorations": text_decorations,
            "spellcheck": spellcheck
        }
        
        if freshness:
            params["freshness"] = freshness
        
        for attempt in range(self.max_retries):
            try:
                response = requests.get(
                    f"{self.base_url}/web/search",
                    headers=self.headers,
                    params=params,
                    timeout=10
                )
                response.raise_for_status()
                result = response.json()
                
                # Cache the result
                self.cache[cache_key] = (result, time.time())
                
                return result
                
            except requests.exceptions.RequestException as e:
                logger.warning(f"Search attempt {attempt + 1} failed: {e}")
                if attempt == self.max_retries - 1:
                    raise
                time.sleep(2 ** attempt)  # Exponential backoff
    
    def get_snippets(self, search_results: Dict[str, Any]) -> List[Dict[str, str]]:
        """Extract and format snippets from search results"""
        snippets = []
        
        if "web" in search_results and "results" in search_results["web"]:
            for result in search_results["web"]["results"]:
                snippet = {
                    "title": result.get("title", ""),
                    "url": result.get("url", ""),
                    "description": result.get("description", ""),
                    "age": result.get("age", ""),
                    "language": result.get("language", "")
                }
                
                # Add extra snippets if available
                if "extra_snippets" in result:
                    snippet["extra_snippets"] = result["extra_snippets"]
                
                snippets.append(snippet)
        
        return snippets
    
    def get_news(self, search_results: Dict[str, Any]) -> List[Dict[str, str]]:
        """Extract news results if available"""
        news = []
        
        if "news" in search_results and "results" in search_results["news"]:
            for result in search_results["news"]["results"]:
                news.append({
                    "title": result.get("title", ""),
                    "url": result.get("url", ""),
                    "description": result.get("description", ""),
                    "age": result.get("age", ""),
                    "source": result.get("meta_url", {}).get("hostname", "")
                })
        
        return news

# Initialize Brave Search client
brave_client = BraveSearchClient(BRAVE_API_KEY)
console.print("[green]✓ Brave Search client initialized[/green]")

## 4. Data Models and State Management

In [ ]:
# Data Models
class SearchResult(BaseModel):
    """Model for search results"""
    query: str
    title: str
    url: str
    description: str
    snippet: Optional[str] = None
    relevance_score: float = Field(default=0.0, ge=0.0, le=1.0)
    timestamp: datetime = Field(default_factory=lambda: datetime.now(timezone.utc))
    source_type: Literal["web", "news", "video"] = "web"
    
class Fact(BaseModel):
    """Model for extracted facts"""
    statement: str
    source_url: str
    confidence: float = Field(ge=0.0, le=1.0)
    supporting_evidence: List[str] = Field(default_factory=list)
    contradicting_evidence: List[str] = Field(default_factory=list)
    verification_status: Literal["verified", "disputed", "unverified"] = "unverified"
    
class AgentResponse(BaseModel):
    """Model for agent responses"""
    agent_name: str
    content: str
    search_queries: List[str] = Field(default_factory=list)
    sources: List[str] = Field(default_factory=list)
    facts: List[Fact] = Field(default_factory=list)
    confidence: float = Field(ge=0.0, le=1.0)
    reasoning: Optional[str] = None

# State for multi-agent workflow
class AgentState(TypedDict):
    """State management for multi-agent workflow"""
    messages: Annotated[Sequence[BaseMessage], add_messages]
    query: str
    search_results: List[SearchResult]
    facts: List[Fact]
    agent_responses: Dict[str, AgentResponse]
    current_agent: str
    iteration: int
    max_iterations: int
    final_response: Optional[str]
    metadata: Dict[str, Any]

console.print("[green]✓ Data models defined[/green]")

## 5. Agent Implementations

In [ ]:
# Base Agent Class
class BaseAgent:
    """Base class for all agents"""
    
    def __init__(self, name: str, llm, brave_client: BraveSearchClient):
        self.name = name
        self.llm = llm
        self.brave_client = brave_client
        self.logger = logging.getLogger(f"{__name__}.{name}")
    
    async def process(self, state: AgentState) -> AgentState:
        """Process state - to be implemented by subclasses"""
        raise NotImplementedError
    
    def search(self, query: str, count: int = 5, freshness: Optional[str] = None) -> List[SearchResult]:
        """Execute search and return formatted results"""
        try:
            results = self.brave_client.web_search(query, count=count, freshness=freshness)
            snippets = self.brave_client.get_snippets(results)
            
            search_results = []
            for snippet in snippets:
                search_results.append(SearchResult(
                    query=query,
                    title=snippet["title"],
                    url=snippet["url"],
                    description=snippet["description"],
                    snippet=" ".join(snippet.get("extra_snippets", [])),
                    relevance_score=0.8  # Default score
                ))
            
            return search_results
            
        except Exception as e:
            self.logger.error(f"Search failed: {e}")
            return []

In [ ]:
# Research Agent
class ResearchAgent(BaseAgent):
    """Agent responsible for gathering information through searches"""
    
    async def process(self, state: AgentState) -> AgentState:
        self.logger.info(f"Research Agent processing query: {state['query']}")
        
        # Generate search queries
        search_prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content="""You are a research agent that generates comprehensive search queries.
            Given a user query, generate 3-5 diverse search queries to gather comprehensive information.
            Consider different perspectives, related topics, and fact-checking needs.
            Return JSON with 'queries' list."""),
            HumanMessage(content=f"Generate search queries for: {state['query']}")
        ])
        
        response = self.llm.invoke(search_prompt.format_messages())
        queries_data = json.loads(response.content)
        
        # Execute searches
        all_results = []
        for query in queries_data["queries"]:
            results = self.search(query, count=5)
            all_results.extend(results)
            console.print(f"[blue]Searched: {query} - Found {len(results)} results[/blue]")
        
        # Update state
        state["search_results"].extend(all_results)
        
        # Create agent response
        agent_response = AgentResponse(
            agent_name=self.name,
            content=f"Gathered {len(all_results)} search results from {len(queries_data['queries'])} queries",
            search_queries=queries_data["queries"],
            sources=[r.url for r in all_results],
            confidence=0.9
        )
        
        state["agent_responses"][self.name] = agent_response
        state["messages"].append(AIMessage(content=agent_response.content, name=self.name))
        
        return state

In [ ]:
# Fact-Checking Agent
class FactCheckingAgent(BaseAgent):
    """Agent responsible for verifying claims and facts"""
    
    async def process(self, state: AgentState) -> AgentState:
        self.logger.info("Fact-Checking Agent processing")
        
        # Extract claims to verify
        extract_prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content="""You are a fact-checking agent.
            Extract key factual claims from the search results that need verification.
            For each claim, assess its verifiability and importance.
            Return JSON with 'claims' list, each having 'statement' and 'importance'."""),
            HumanMessage(content=f"""Query: {state['query']}
            
Search results:
{json.dumps([r.dict() for r in state['search_results'][:10]], indent=2, default=str)}""")
        ])
        
        response = self.llm.invoke(extract_prompt.format_messages())
        claims_data = json.loads(response.content)
        
        # Verify each claim
        verified_facts = []
        for claim in claims_data["claims"][:5]:  # Limit to top 5 claims
            # Search for verification
            verification_query = f"fact check {claim['statement']}"
            verification_results = self.search(verification_query, count=3)
            
            # Analyze verification results
            verify_prompt = ChatPromptTemplate.from_messages([
                SystemMessage(content="""Analyze the search results to verify the claim.
                Determine if the claim is verified, disputed, or unverified.
                Return JSON with 'status', 'confidence', 'supporting_evidence', 'contradicting_evidence'."""),
                HumanMessage(content=f"""Claim: {claim['statement']}
                
Verification results:
{json.dumps([r.dict() for r in verification_results], indent=2, default=str)}""")
            ])
            
            verify_response = self.llm.invoke(verify_prompt.format_messages())
            verification_data = json.loads(verify_response.content)
            
            fact = Fact(
                statement=claim["statement"],
                source_url=verification_results[0].url if verification_results else "",
                confidence=verification_data["confidence"],
                supporting_evidence=verification_data["supporting_evidence"],
                contradicting_evidence=verification_data["contradicting_evidence"],
                verification_status=verification_data["status"]
            )
            
            verified_facts.append(fact)
            console.print(f"[yellow]Verified: {claim['statement'][:50]}... - Status: {fact.verification_status}[/yellow]")
        
        # Update state
        state["facts"].extend(verified_facts)
        
        # Create agent response
        agent_response = AgentResponse(
            agent_name=self.name,
            content=f"Verified {len(verified_facts)} facts",
            facts=verified_facts,
            confidence=0.85
        )
        
        state["agent_responses"][self.name] = agent_response
        state["messages"].append(AIMessage(content=agent_response.content, name=self.name))
        
        return state

In [ ]:
# Analysis Agent
class AnalysisAgent(BaseAgent):
    """Agent responsible for analyzing and synthesizing information"""
    
    async def process(self, state: AgentState) -> AgentState:
        self.logger.info("Analysis Agent processing")
        
        # Synthesize information
        synthesis_prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content="""You are an analysis agent that synthesizes information.
            Analyze the search results and verified facts to provide comprehensive insights.
            Identify patterns, contradictions, and key findings.
            Provide a structured analysis with main points, supporting evidence, and caveats.
            Return JSON with 'analysis', 'key_findings', 'contradictions', 'confidence'."""),
            HumanMessage(content=f"""Query: {state['query']}

Search Results Summary:
{json.dumps([{'title': r.title, 'description': r.description} for r in state['search_results'][:10]], indent=2)}

Verified Facts:
{json.dumps([f.dict() for f in state['facts']], indent=2, default=str)}""")
        ])
        
        response = self.llm.invoke(synthesis_prompt.format_messages())
        analysis_data = json.loads(response.content)
        
        # Additional deep-dive if needed
        if analysis_data["confidence"] < 0.7:
            # Perform additional searches for unclear areas
            console.print("[orange]Low confidence - performing additional research[/orange]")
            
            for finding in analysis_data["key_findings"][:2]:
                additional_results = self.search(f"explain {finding}", count=3)
                state["search_results"].extend(additional_results)
        
        # Create comprehensive response
        agent_response = AgentResponse(
            agent_name=self.name,
            content=analysis_data["analysis"],
            sources=[r.url for r in state["search_results"][:5]],
            confidence=analysis_data["confidence"],
            reasoning=f"Key findings: {', '.join(analysis_data['key_findings'])}"
        )
        
        state["agent_responses"][self.name] = agent_response
        state["messages"].append(AIMessage(content=agent_response.content, name=self.name))
        
        return state

In [ ]:
# Quality Assurance Agent
class QualityAssuranceAgent(BaseAgent):
    """Agent responsible for ensuring quality and completeness"""
    
    async def process(self, state: AgentState) -> AgentState:
        self.logger.info("QA Agent processing")
        
        # Evaluate completeness and quality
        qa_prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content="""You are a quality assurance agent.
            Evaluate the completeness and accuracy of the gathered information.
            Check for:
            1. Answer completeness - does it fully address the query?
            2. Source reliability - are sources credible?
            3. Fact accuracy - are facts properly verified?
            4. Coherence - is the information consistent?
            5. Missing information - what gaps exist?
            
            Return JSON with 'quality_score', 'completeness_score', 'issues', 'missing_info', 'recommendations'."""),
            HumanMessage(content=f"""Query: {state['query']}

Agent Responses:
{json.dumps({name: resp.dict() for name, resp in state['agent_responses'].items()}, indent=2, default=str)}""")
        ])
        
        response = self.llm.invoke(qa_prompt.format_messages())
        qa_data = json.loads(response.content)
        
        # If quality is low, trigger additional research
        if qa_data["quality_score"] < 0.7 or qa_data["completeness_score"] < 0.7:
            console.print("[red]Quality check failed - requesting additional research[/red]")
            
            for missing in qa_data["missing_info"][:2]:
                additional_results = self.search(missing, count=3, freshness="day")
                state["search_results"].extend(additional_results)
            
            state["iteration"] += 1
        
        # Generate final assessment
        final_prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content="""Generate a comprehensive, well-structured response that:
            1. Directly answers the user's query
            2. Includes verified facts with citations
            3. Acknowledges any uncertainties or contradictions
            4. Provides actionable insights where relevant
            
            Format the response in clear sections with proper citations."""),
            HumanMessage(content=f"""Query: {state['query']}

All gathered information:
{json.dumps({name: resp.dict() for name, resp in state['agent_responses'].items()}, indent=2, default=str)}

Quality Assessment:
{json.dumps(qa_data, indent=2)}""")
        ])
        
        final_response = self.llm.invoke(final_prompt.format_messages())
        
        # Update state with final response
        state["final_response"] = final_response.content
        state["metadata"]["qa_assessment"] = qa_data
        
        agent_response = AgentResponse(
            agent_name=self.name,
            content=f"Quality Score: {qa_data['quality_score']}, Completeness: {qa_data['completeness_score']}",
            confidence=(qa_data["quality_score"] + qa_data["completeness_score"]) / 2
        )
        
        state["agent_responses"][self.name] = agent_response
        state["messages"].append(AIMessage(content=final_response.content, name=self.name))
        
        return state

console.print("[green]✓ All agents defined[/green]")

## 6. Multi-Agent Orchestrator

In [ ]:
class MultiAgentOrchestrator:
    """Orchestrates the multi-agent workflow"""
    
    def __init__(self, llm_model: str = "gpt-4"):
        # Initialize LLMs
        self.primary_llm = ChatOpenAI(model=llm_model, temperature=0.3)
        self.fast_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)
        
        # Initialize Brave client
        self.brave_client = brave_client
        
        # Initialize agents
        self.agents = {
            "research": ResearchAgent("ResearchAgent", self.primary_llm, self.brave_client),
            "fact_check": FactCheckingAgent("FactCheckAgent", self.fast_llm, self.brave_client),
            "analysis": AnalysisAgent("AnalysisAgent", self.primary_llm, self.brave_client),
            "qa": QualityAssuranceAgent("QAAgent", self.primary_llm, self.brave_client)
        }
        
        # Build the graph
        self.graph = self._build_graph()
    
    def _build_graph(self) -> StateGraph:
        """Build the LangGraph workflow"""
        
        # Create the graph
        workflow = StateGraph(AgentState)
        
        # Add nodes
        workflow.add_node("research", self.research_node)
        workflow.add_node("fact_check", self.fact_check_node)
        workflow.add_node("analysis", self.analysis_node)
        workflow.add_node("qa", self.qa_node)
        workflow.add_node("router", self.router_node)
        
        # Set entry point
        workflow.set_entry_point("research")
        
        # Add edges
        workflow.add_edge("research", "fact_check")
        workflow.add_edge("fact_check", "analysis")
        workflow.add_edge("analysis", "qa")
        workflow.add_edge("qa", "router")
        
        # Add conditional edges from router
        workflow.add_conditional_edges(
            "router",
            self.should_continue,
            {
                "research": "research",
                "end": END
            }
        )
        
        return workflow.compile()
    
    async def research_node(self, state: AgentState) -> AgentState:
        return await self.agents["research"].process(state)
    
    async def fact_check_node(self, state: AgentState) -> AgentState:
        return await self.agents["fact_check"].process(state)
    
    async def analysis_node(self, state: AgentState) -> AgentState:
        return await self.agents["analysis"].process(state)
    
    async def qa_node(self, state: AgentState) -> AgentState:
        return await self.agents["qa"].process(state)
    
    def router_node(self, state: AgentState) -> AgentState:
        """Router node to decide next steps"""
        state["current_agent"] = "router"
        return state
    
    def should_continue(self, state: AgentState) -> str:
        """Decide whether to continue or end the workflow"""
        
        # Check if we have a final response
        if state.get("final_response") and state["iteration"] >= 1:
            return "end"
        
        # Check if we've reached max iterations
        if state["iteration"] >= state["max_iterations"]:
            return "end"
        
        # Check quality scores from QA
        qa_assessment = state.get("metadata", {}).get("qa_assessment", {})
        if qa_assessment:
            quality = qa_assessment.get("quality_score", 0)
            completeness = qa_assessment.get("completeness_score", 0)
            
            if quality >= 0.8 and completeness >= 0.8:
                return "end"
        
        # Continue with more research
        return "research"
    
    async def run(self, query: str, max_iterations: int = 2) -> Dict[str, Any]:
        """Run the multi-agent workflow"""
        
        # Initialize state
        initial_state = {
            "messages": [HumanMessage(content=query)],
            "query": query,
            "search_results": [],
            "facts": [],
            "agent_responses": {},
            "current_agent": "orchestrator",
            "iteration": 0,
            "max_iterations": max_iterations,
            "final_response": None,
            "metadata": {}
        }
        
        console.print(Panel(f"[bold green]Starting multi-agent workflow for:[/bold green]\n{query}", title="Query"))
        
        # Run the workflow
        final_state = await self.graph.ainvoke(initial_state)
        
        return final_state

# Initialize orchestrator
orchestrator = MultiAgentOrchestrator(llm_model="gpt-4")
console.print("[green]✓ Orchestrator initialized[/green]")

## 7. Visualization and Reporting

In [ ]:
def visualize_results(state: Dict[str, Any]):
    """Visualize the workflow results"""
    
    # Display agent contributions
    console.print("\n" + "="*80)
    console.print(Panel("[bold]Multi-Agent Workflow Results[/bold]", style="cyan"))
    
    # Agent responses table
    table = Table(title="Agent Contributions")
    table.add_column("Agent", style="cyan")
    table.add_column("Confidence", style="yellow")
    table.add_column("Sources Used", style="green")
    table.add_column("Facts Verified", style="magenta")
    
    for agent_name, response in state["agent_responses"].items():
        table.add_row(
            agent_name,
            f"{response.confidence:.2f}",
            str(len(response.sources)),
            str(len(response.facts))
        )
    
    console.print(table)
    
    # Display verified facts
    if state["facts"]:
        console.print("\n[bold]Verified Facts:[/bold]")
        for i, fact in enumerate(state["facts"], 1):
            status_color = {
                "verified": "green",
                "disputed": "red",
                "unverified": "yellow"
            }.get(fact.verification_status, "white")
            
            console.print(f"  {i}. [{status_color}]{fact.statement}[/{status_color}]")
            console.print(f"     Confidence: {fact.confidence:.2f} | Source: {fact.source_url[:50]}...")
    
    # Display final response
    if state.get("final_response"):
        console.print("\n" + "="*80)
        console.print(Panel(state["final_response"], title="[bold green]Final Response[/bold green]", border_style="green"))
    
    # Display QA assessment
    if "qa_assessment" in state.get("metadata", {}):
        qa = state["metadata"]["qa_assessment"]
        console.print("\n[bold]Quality Assessment:[/bold]")
        console.print(f"  Quality Score: {qa.get('quality_score', 0):.2f}")
        console.print(f"  Completeness: {qa.get('completeness_score', 0):.2f}")
        
        if qa.get("issues"):
            console.print("  Issues found:")
            for issue in qa["issues"]:
                console.print(f"    - {issue}")

def export_results(state: Dict[str, Any], filename: str = "results.json"):
    """Export results to JSON file"""
    
    export_data = {
        "query": state["query"],
        "timestamp": datetime.now(timezone.utc).isoformat(),
        "iterations": state["iteration"],
        "agent_responses": {name: resp.dict() for name, resp in state["agent_responses"].items()},
        "facts": [fact.dict() for fact in state["facts"]],
        "search_results_count": len(state["search_results"]),
        "final_response": state.get("final_response"),
        "metadata": state["metadata"]
    }
    
    with open(filename, "w") as f:
        json.dump(export_data, f, indent=2, default=str)
    
    console.print(f"\n[green]Results exported to {filename}[/green]")

console.print("[green]✓ Visualization functions ready[/green]")

## 8. Example Usage

In [ ]:
# Example 1: Technology Research Query
async def example_tech_research():
    query = "What are the latest developments in quantum computing and how do they compare to classical computing for machine learning tasks?"
    
    result = await orchestrator.run(query, max_iterations=2)
    visualize_results(result)
    export_results(result, "quantum_computing_research.json")
    
    return result

# Run the example
result = await example_tech_research()

In [ ]:
# Example 2: Current Events Analysis
async def example_current_events():
    query = "What are the major AI safety initiatives launched in 2024 and their potential impact on AI development?"
    
    result = await orchestrator.run(query, max_iterations=2)
    visualize_results(result)
    export_results(result, "ai_safety_2024.json")
    
    return result

# Run the example
result = await example_current_events()

In [ ]:
# Example 3: Comparative Analysis
async def example_comparative_analysis():
    query = "Compare the approaches of OpenAI, Anthropic, and Google DeepMind to AI alignment and safety research"
    
    result = await orchestrator.run(query, max_iterations=3)
    visualize_results(result)
    export_results(result, "ai_companies_comparison.json")
    
    return result

# Run the example
result = await example_comparative_analysis()

## 9. Advanced Features

In [ ]:
# Stream Processing for Real-time Updates
class StreamingOrchestrator(MultiAgentOrchestrator):
    """Extended orchestrator with streaming capabilities"""
    
    async def run_with_streaming(self, query: str, max_iterations: int = 2):
        """Run workflow with real-time streaming updates"""
        
        initial_state = {
            "messages": [HumanMessage(content=query)],
            "query": query,
            "search_results": [],
            "facts": [],
            "agent_responses": {},
            "current_agent": "orchestrator",
            "iteration": 0,
            "max_iterations": max_iterations,
            "final_response": None,
            "metadata": {}
        }
        
        console.print(Panel(f"[bold green]Streaming workflow for:[/bold green]\n{query}", title="Query"))
        
        # Stream events
        async for event in self.graph.astream_events(initial_state, version="v1"):
            if event["event"] == "on_chain_start":
                console.print(f"[dim]Starting: {event['name']}[/dim]")
            elif event["event"] == "on_chain_end":
                console.print(f"[dim]Completed: {event['name']}[/dim]")
            elif event["event"] == "on_chat_model_stream":
                # Stream token-by-token for supported models
                if event.get("data", {}).get("chunk"):
                    print(event["data"]["chunk"].content, end="", flush=True)
        
        # Get final state
        final_state = await self.graph.ainvoke(initial_state)
        return final_state

# Caching layer for improved performance
class CachedBraveSearch(BraveSearchClient):
    """Brave Search with persistent caching"""
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.persistent_cache = {}
        self.load_cache()
    
    def load_cache(self, filename="search_cache.json"):
        """Load cache from file"""
        try:
            with open(filename, "r") as f:
                self.persistent_cache = json.load(f)
        except FileNotFoundError:
            self.persistent_cache = {}
    
    def save_cache(self, filename="search_cache.json"):
        """Save cache to file"""
        with open(filename, "w") as f:
            json.dump(self.persistent_cache, f, indent=2)

console.print("[green]✓ Advanced features loaded[/green]")

## 10. Performance Metrics and Monitoring

In [ ]:
import time
from collections import defaultdict

class PerformanceMonitor:
    """Monitor and track agent performance"""
    
    def __init__(self):
        self.metrics = defaultdict(list)
        self.timings = defaultdict(list)
    
    def track_agent_performance(self, agent_name: str, start_time: float, end_time: float, 
                               success: bool, confidence: float):
        """Track individual agent performance"""
        duration = end_time - start_time
        
        self.timings[agent_name].append(duration)
        self.metrics[agent_name].append({
            "duration": duration,
            "success": success,
            "confidence": confidence,
            "timestamp": datetime.now(timezone.utc).isoformat()
        })
    
    def get_statistics(self):
        """Calculate performance statistics"""
        stats = {}
        
        for agent_name, metrics_list in self.metrics.items():
            durations = [m["duration"] for m in metrics_list]
            confidences = [m["confidence"] for m in metrics_list]
            success_rate = sum(m["success"] for m in metrics_list) / len(metrics_list)
            
            stats[agent_name] = {
                "avg_duration": sum(durations) / len(durations),
                "min_duration": min(durations),
                "max_duration": max(durations),
                "avg_confidence": sum(confidences) / len(confidences),
                "success_rate": success_rate,
                "total_runs": len(metrics_list)
            }
        
        return stats
    
    def display_dashboard(self):
        """Display performance dashboard"""
        stats = self.get_statistics()
        
        table = Table(title="Agent Performance Dashboard")
        table.add_column("Agent", style="cyan")
        table.add_column("Avg Duration (s)", style="yellow")
        table.add_column("Avg Confidence", style="green")
        table.add_column("Success Rate", style="magenta")
        table.add_column("Total Runs", style="blue")
        
        for agent_name, agent_stats in stats.items():
            table.add_row(
                agent_name,
                f"{agent_stats['avg_duration']:.2f}",
                f"{agent_stats['avg_confidence']:.2f}",
                f"{agent_stats['success_rate']:.0%}",
                str(agent_stats['total_runs'])
            )
        
        console.print(table)

# Initialize performance monitor
monitor = PerformanceMonitor()
console.print("[green]✓ Performance monitoring initialized[/green]")

## 11. Error Handling and Recovery

In [ ]:
class RobustOrchestrator(MultiAgentOrchestrator):
    """Orchestrator with enhanced error handling and recovery"""
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.fallback_strategies = {
            "search_failure": self.handle_search_failure,
            "llm_failure": self.handle_llm_failure,
            "parsing_failure": self.handle_parsing_failure
        }
    
    async def handle_search_failure(self, state: AgentState, error: Exception) -> AgentState:
        """Handle search API failures"""
        console.print(f"[red]Search failure: {error}[/red]")
        
        # Try alternative search strategies
        state["metadata"]["search_errors"] = state["metadata"].get("search_errors", []) + [str(error)]
        
        # Use cached results if available
        if hasattr(self.brave_client, 'cache') and self.brave_client.cache:
            console.print("[yellow]Using cached search results[/yellow]")
            # Return state with cached results
        
        return state
    
    async def handle_llm_failure(self, state: AgentState, error: Exception) -> AgentState:
        """Handle LLM API failures"""
        console.print(f"[red]LLM failure: {error}[/red]")
        
        # Try with fallback model
        state["metadata"]["llm_errors"] = state["metadata"].get("llm_errors", []) + [str(error)]
        
        # Use simpler prompts or cached responses
        return state
    
    async def handle_parsing_failure(self, state: AgentState, error: Exception) -> AgentState:
        """Handle JSON parsing failures"""
        console.print(f"[red]Parsing failure: {error}[/red]")
        
        # Try alternative parsing strategies
        state["metadata"]["parsing_errors"] = state["metadata"].get("parsing_errors", []) + [str(error)]
        
        return state
    
    async def run_with_recovery(self, query: str, max_iterations: int = 2) -> Dict[str, Any]:
        """Run workflow with automatic error recovery"""
        
        try:
            return await self.run(query, max_iterations)
        except Exception as e:
            console.print(f"[bold red]Workflow failed: {e}[/bold red]")
            
            # Attempt recovery
            for strategy_name, strategy_func in self.fallback_strategies.items():
                try:
                    console.print(f"[yellow]Attempting recovery: {strategy_name}[/yellow]")
                    # Recovery logic here
                    return await self.run(query, max_iterations=1)  # Simplified retry
                except Exception as recovery_error:
                    console.print(f"[red]Recovery failed: {recovery_error}[/red]")
            
            raise

console.print("[green]✓ Error handling configured[/green]")

## 12. Interactive Demo

In [ ]:
async def interactive_demo():
    """Interactive demonstration of the multi-agent system"""
    
    console.print(Panel(
        "[bold cyan]Multi-Agent Brave Search Workflow Demo[/bold cyan]\n\n"
        "This system uses multiple specialized agents to:\n"
        "• Research information using Brave Search API\n"
        "• Verify facts and claims\n"
        "• Analyze and synthesize findings\n"
        "• Ensure quality and completeness\n",
        title="Welcome",
        border_style="cyan"
    ))
    
    # Predefined queries for demonstration
    demo_queries = [
        "What are the latest breakthroughs in large language models and their implications for AGI?",
        "Compare the environmental impact of electric vehicles vs hydrogen fuel cells",
        "What are the emerging cybersecurity threats in 2024 and recommended defenses?",
        "Analyze the current state of quantum computing startups and their funding"
    ]
    
    console.print("\n[bold]Select a demo query:[/bold]")
    for i, query in enumerate(demo_queries, 1):
        console.print(f"  {i}. {query}")
    
    # For notebook, we'll use the first query
    selected_query = demo_queries[0]
    console.print(f"\n[green]Selected: {selected_query}[/green]\n")
    
    # Run the workflow
    start_time = time.time()
    result = await orchestrator.run(selected_query, max_iterations=2)
    end_time = time.time()
    
    # Display results
    visualize_results(result)
    
    # Show performance metrics
    console.print(f"\n[bold]Performance Summary:[/bold]")
    console.print(f"  Total time: {end_time - start_time:.2f} seconds")
    console.print(f"  Search queries executed: {len(result['search_results'])}")
    console.print(f"  Facts verified: {len(result['facts'])}")
    console.print(f"  Agents involved: {len(result['agent_responses'])}")
    
    # Export results
    export_results(result, "demo_results.json")
    
    return result

# Run the interactive demo
demo_result = await interactive_demo()

## 13. Best Practices and Optimization Tips

### Key Recommendations:

1. **API Rate Limiting**: Implement proper rate limiting for Brave Search API
2. **Caching Strategy**: Use intelligent caching to reduce API calls and costs
3. **Prompt Engineering**: Optimize prompts for each agent's specific role
4. **Error Recovery**: Implement comprehensive error handling and fallback strategies
5. **Monitoring**: Track performance metrics to identify bottlenecks
6. **Grounding Quality**: Always verify information from multiple sources
7. **Context Management**: Efficiently manage context size for LLM calls

In [ ]:
# Configuration best practices
BEST_PRACTICE_CONFIG = {
    "brave_search": {
        "max_results_per_query": 10,
        "cache_ttl": 3600,  # 1 hour
        "rate_limit": 100,  # requests per minute
        "retry_attempts": 3,
        "timeout": 10
    },
    "agents": {
        "research": {
            "max_queries": 5,
            "query_diversity": True,
            "include_news": True
        },
        "fact_check": {
            "min_sources": 2,
            "confidence_threshold": 0.7
        },
        "analysis": {
            "synthesis_depth": "comprehensive",
            "include_contradictions": True
        },
        "qa": {
            "quality_threshold": 0.8,
            "completeness_threshold": 0.8
        }
    },
    "workflow": {
        "max_iterations": 3,
        "parallel_processing": False,  # Can be enabled for faster processing
        "streaming": True,
        "save_intermediate_results": True
    }
}

console.print(Panel(
    "[green]✓ Multi-Agent Brave Search Workflow Setup Complete![/green]\n\n"
    "The system is now ready to:\n"
    "• Execute complex research queries\n"
    "• Verify information through multiple sources\n"
    "• Provide grounded, fact-checked responses\n"
    "• Track performance and optimize over time",
    title="Setup Complete",
    border_style="green"
))

## Conclusion

This notebook demonstrates a production-ready multi-agent system using Brave Search API for grounding. The system features:

- **Modular Architecture**: Easily extensible with new agents
- **Robust Search Integration**: Comprehensive Brave Search API implementation
- **Fact Verification**: Multi-source verification for accuracy
- **Quality Assurance**: Built-in quality checks and iterative refinement
- **Performance Monitoring**: Track and optimize agent performance
- **Error Recovery**: Graceful handling of failures

The workflow can be adapted for various use cases including:
- Research automation
- Real-time fact-checking
- Content generation with citations
- Competitive intelligence gathering
- News aggregation and analysis